#  回归树

In [33]:
from numpy import *

In [34]:
def loadDataSet(fileName):
    '''
    返回二维列表，按行嵌套
    '''
    dataMat = []
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = list(map(float, curLine))  #map函数，在python3里返回的是迭代器(function, iterable)
        dataMat.append(fltLine)
    return dataMat

def binSplitDataSet(dataSet, feature, value):
    '''
    给定特征和特征值，切分数据集合,输入是矩阵,切分成两个数据矩阵
    '''
    mat0 = dataSet[(dataSet[:, feature] > value).T.tolist()[0], :]
    mat1 = dataSet[(dataSet[:, feature] <= value).T.tolist()[0], :]
    return mat0, mat1

In [35]:
testMat = mat(eye((4)))

In [36]:
testMat

matrix([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [37]:
mat0, mat1 = binSplitDataSet(testMat, 1, 0.5)

In [38]:
mat0

matrix([[0., 1., 0., 0.]])

In [39]:
mat1

matrix([[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [40]:
def regLeaf(dataSet):
    return mean(dataSet[:, -1])

In [41]:
def regErr(dataSet):
    return var(dataSet[:, -1]) * shape(dataSet)[0]  #总方差

In [42]:
def chooseBestSplit(dataSet, leafType=regLeaf, errType=regErr, ops=(1, 4)):
    tolS = ops[0]
    tolN = ops[1]
    if len(set(dataSet[:, -1].T.tolist()[0])) == 1:  #所有特征值相同，退出
        return None, leafType(dataSet)
    m, n = shape(dataSet)
    S = errType(dataSet)
    bestS = inf
    bestIndex = 0
    bestValue = 0
    for featIndex in range(n-1):
        for splitVal in set(dataSet[:, featIndex].T.tolist()[0]): #先转置
            mat0, mat1 = binSplitDataSet(dataSet, featIndex, splitVal)
            if (shape(mat0)[0] < tolN) or (shape(mat1)[0] < tolN):
                continue
            newS = errType(mat0) + errType(mat1)
            if newS < bestS:
                bestIndex = featIndex
                bestValue = splitVal
                bestS = newS
    if (S - bestS) < tolS:  #误差变化不大，退出
        return None, leafType(dataSet)
    mat0, mat1 = binSplitDataSet(dataSet, bestIndex, bestValue)
    if (shape(mat0)[0] < tolN) or (shape(mat1)[0] < tolN):  #切分后子集太小，退出
        return None, leafType(dataSet)
    return bestIndex, bestValue

In [43]:
def createTree(dataSet, leafType=regLeaf, errType=regErr, ops=(1, 4)):
    feat, val = chooseBestSplit(dataSet, leafType, errType, ops)
    if feat == None:
        return val
    retTree = {}
    retTree['spInd'] = feat
    retTree['spVal'] = val
    lSet, rSet = binSplitDataSet(dataSet, feat, val)
    retTree['left'] = createTree(lSet, leafType, errType, ops)
    retTree['right'] = createTree(rSet, leafType, errType, ops)
    return retTree

In [44]:
myDat = loadDataSet('ex00.txt')
myDat = mat(myDat)
print(myDat)
createTree(myDat)

[[ 3.609800e-02  1.550960e-01]
 [ 9.933490e-01  1.077553e+00]
 [ 5.308970e-01  8.934620e-01]
 [ 7.123860e-01  5.648580e-01]
 [ 3.435540e-01 -3.717000e-01]
 [ 9.801600e-02 -3.327600e-01]
 [ 6.911150e-01  8.343910e-01]
 [ 9.135800e-02  9.993500e-02]
 [ 7.270980e-01  1.000567e+00]
 [ 9.519490e-01  9.452550e-01]
 [ 7.685960e-01  7.602190e-01]
 [ 5.413140e-01  8.937480e-01]
 [ 1.463660e-01  3.428300e-02]
 [ 6.731950e-01  9.150770e-01]
 [ 1.835100e-01  1.848430e-01]
 [ 3.395630e-01  2.067830e-01]
 [ 5.179210e-01  1.493586e+00]
 [ 7.037550e-01  1.101678e+00]
 [ 8.307000e-03  6.997600e-02]
 [ 2.439090e-01 -2.946700e-02]
 [ 3.069640e-01 -1.773210e-01]
 [ 3.649200e-02  4.081550e-01]
 [ 2.955110e-01  2.882000e-03]
 [ 8.375220e-01  1.229373e+00]
 [ 2.020540e-01 -8.774400e-02]
 [ 9.193840e-01  1.029889e+00]
 [ 3.772010e-01 -2.435500e-01]
 [ 8.148250e-01  1.095206e+00]
 [ 6.112700e-01  9.820360e-01]
 [ 7.224300e-02 -4.209830e-01]
 [ 4.102300e-01  3.317220e-01]
 [ 8.690770e-01  1.114825e+00]
 [ 6.205

{'spInd': 0,
 'spVal': 0.48813,
 'left': 1.0180967672413792,
 'right': -0.04465028571428572}

In [45]:
myDat1 = loadDataSet('ex0.txt')
myMat1 = mat(myDat1)
createTree(myMat1)

{'spInd': 1,
 'spVal': 0.39435,
 'left': {'spInd': 1,
  'spVal': 0.582002,
  'left': {'spInd': 1,
   'spVal': 0.797583,
   'left': 3.9871632,
   'right': 2.9836209534883724},
  'right': 1.980035071428571},
 'right': {'spInd': 1,
  'spVal': 0.197834,
  'left': 1.0289583666666666,
  'right': -0.023838155555555553}}

In [46]:
myDat2 = loadDataSet('ex2.txt')
myMat2 = mat(myDat2)
createTree(myMat2)

{'spInd': 0,
 'spVal': 0.499171,
 'left': {'spInd': 0,
  'spVal': 0.729397,
  'left': {'spInd': 0,
   'spVal': 0.952833,
   'left': {'spInd': 0,
    'spVal': 0.958512,
    'left': 105.24862350000001,
    'right': 112.42895575000001},
   'right': {'spInd': 0,
    'spVal': 0.759504,
    'left': {'spInd': 0,
     'spVal': 0.790312,
     'left': {'spInd': 0,
      'spVal': 0.833026,
      'left': {'spInd': 0,
       'spVal': 0.944221,
       'left': 87.3103875,
       'right': {'spInd': 0,
        'spVal': 0.85497,
        'left': {'spInd': 0,
         'spVal': 0.910975,
         'left': 96.452867,
         'right': {'spInd': 0,
          'spVal': 0.892999,
          'left': 104.825409,
          'right': {'spInd': 0,
           'spVal': 0.872883,
           'left': 95.181793,
           'right': 102.25234449999999}}},
        'right': 95.27584316666666}},
      'right': {'spInd': 0,
       'spVal': 0.811602,
       'left': 81.110152,
       'right': 88.78449880000001}},
     'right': 102.

In [47]:
createTree(myMat2, ops=(10000, 4))

{'spInd': 0,
 'spVal': 0.499171,
 'left': 101.35815937735848,
 'right': -2.637719329787234}

## 提前设定ops值是预剪枝，而下面的后剪枝是更理想化的剪枝方法

In [48]:
def isTree(obj):
    return (type(obj).__name__ == "dict")

def getMean(tree):
    if isTree(tree['right']):
        tree['right'] = getMean(tree['right'])
    if isTree(tree['left']):
        tree['left'] = getMean(tree['left'])
    return (tree['left'] + tree['right']) / 2.0

def prune(tree, testData):
    if shape(testData)[0] == 0:
        return getMean(tree)
    if (isTree(tree['right']) or  isTree(tree['left'])):
        lSet, rSet = binSplitDataSet(testData, tree['spInd'], tree['spVal'])
    if isTree(tree['left']):
        tree['left'] = prune(tree['left'], lSet)
    if isTree(tree['right']):
        tree['right'] = prune(tree['right'], rSet)
    if not isTree(tree['left']) and not isTree(tree['right']):
        lSet, rSet = binSplitDataSet(testData, tree['spInd'], tree['spVal'])
        errorNoMerge = sum(power(lSet[:, -1] - tree['left'], 2)) + sum(power(rSet[:, -1] - tree['right'], 2))
        treeMean = (tree['left'] + tree['right']) / 2.0
        errorMerge = sum(power(testData[:, -1] - treeMean, 2))
        if errorMerge < errorNoMerge:
            print('merging')
            return treeMean
        else:
            return tree
    else:
        return tree

In [49]:
myTree = createTree(myMat2, ops=(0, 1))

In [50]:
myDatTest = loadDataSet('ex2test.txt')
myMat2Test = mat(myDatTest)
prune(myTree, myMat2Test)

merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging


{'spInd': 0,
 'spVal': 0.499171,
 'left': {'spInd': 0,
  'spVal': 0.729397,
  'left': {'spInd': 0,
   'spVal': 0.952833,
   'left': {'spInd': 0,
    'spVal': 0.965969,
    'left': 92.5239915,
    'right': {'spInd': 0,
     'spVal': 0.956951,
     'left': {'spInd': 0,
      'spVal': 0.958512,
      'left': {'spInd': 0,
       'spVal': 0.960398,
       'left': 112.386764,
       'right': 123.559747},
      'right': 135.837013},
     'right': 111.2013225}},
   'right': {'spInd': 0,
    'spVal': 0.759504,
    'left': {'spInd': 0,
     'spVal': 0.763328,
     'left': {'spInd': 0,
      'spVal': 0.769043,
      'left': {'spInd': 0,
       'spVal': 0.790312,
       'left': {'spInd': 0,
        'spVal': 0.806158,
        'left': {'spInd': 0,
         'spVal': 0.815215,
         'left': {'spInd': 0,
          'spVal': 0.833026,
          'left': {'spInd': 0,
           'spVal': 0.841547,
           'left': {'spInd': 0,
            'spVal': 0.841625,
            'left': {'spInd': 0,
            

# 模型树

In [51]:
def linearSolve(dataSet):
    m, n = shape(dataSet)
    X = mat(ones((m, n)))
    Y = mat(ones((m, 1)))
    X[:, 1:n] = dataSet[:, 0:n-1]
    Y = dataSet[:, -1]
    xTx = X.T * X
    if linalg.det(xTx) == 0.0:
        raise NameError('奇异矩阵，不能求逆，尝试增加ops的第二个值')
    ws = xTx.I * (X.T * Y)
    return ws, X, Y

def modelLeaf(dataSet):
    ws, X, Y = linearSolve(dataSet)
    return ws

def modelErr(dataSet):
    ws, X, Y = linearSolve(dataSet)
    yHat = X * ws
    return sum(power(Y - yHat, 2))


In [52]:
myMat2 = mat(loadDataSet('exp2.txt'))

In [53]:
createTree(myMat2, modelLeaf, modelErr, (1, 10))

{'spInd': 0, 'spVal': 0.285477, 'left': matrix([[1.69855694e-03],
         [1.19647739e+01]]), 'right': matrix([[3.46877936],
         [1.18521743]])}

# 比较树回归和标准回归

In [54]:
def regTreeEval(model, inDat):
    '''
    为了与模型树保持一致
    '''
    return float(model)

def modelTreeEval(model, inDat):
    n = shape(inDat)[1]
    X = mat(ones((1, n+1)))
    X[:, 1:n+1] = inDat
    return float(X * model)

def treeForeCast(tree, inData, modelEval=regTreeEval):
    if not isTree(tree):
        return modelEval(tree, inData)
    if inData[tree['spInd']] > tree['spVal']:
        if isTree(tree['left']):
            return treeForeCast(tree['left'], inData, modelEval)
        else:
            return modelEval(tree['left'], inData)
    else:
        if isTree(tree['right']):
            return treeForeCast(tree['right'], inData, modelEval)
        else:
            return modelEval(tree['right'], inData)
        
def createForeCast(tree, testData, modelEval=regTreeEval):
    m = len(testData)
    yHat = mat(ones((m, 1)))
    for i in range(m):
        yHat[i, 0] = treeForeCast(tree, mat(testData[i]), modelEval)
    return yHat

## 回归树

In [72]:
trainMat = mat(loadDataSet('bikeSpeedVsIq_train.txt'))
testMat = mat(loadDataSet('bikeSpeedVsIq_test.txt'))
myTree = createTree(trainMat, ops=(1, 20))
yHat = createForeCast(myTree, testMat[:, 0])
corrcoef(yHat, testMat[:, 1], rowvar=0)[0, 1]

0.9640852318222141

## 模型树

In [73]:
myTree = createTree(trainMat, modelLeaf, modelErr, (1, 20))
yHat = createForeCast(myTree, testMat[:, 0], modelTreeEval)
corrcoef(yHat, testMat[:, 1], rowvar=0)[0,1]

0.9760412191380593

## 简单线性模型

In [60]:
ws, X, Y = linearSolve(trainMat)
ws

matrix([[37.58916794],
        [ 6.18978355]])

In [74]:
for i in range(shape(testMat)[0]):
    yHat[i] = testMat[i, 0] * ws[1,0] + ws[0, 0]

In [75]:
corrcoef(yHat, testMat[:, 1], rowvar=0)[0, 1]

0.9434684235674763